# Convolutional Neural Networks
---
The reason why CNNs work so well is because of 2 properties observed in data:
* __Stationarity of Statistics:__ This means given small local patches, there is a high probability of finding recurrent patterns in the locality. A simpler definition for this is that some motifs tend to reoccur within the (image) data. This allows for parameter sharing in case of CNNs, which inversely affects the number of parameters in the model, and hence allows relatively lesser training times as compared to, say, fully connected layers.

* __Locality of Pixel Dependencies:__ This principal states that pixels that are close to each other tend to be more correlated and dependent on each other as compared to those far away. In simpler words, pixels that are closer to each other tend to be of similar color. This also means that related data tends to be concentrated into small patches. Locality affects that sparsity of the connections.

* __Compositionality:__ Talking about in terms of image data, images are composed of smaller, simpler patterns. In fact, all data is composed of simpler data. Thus, instead of looking for a certain object within the image, the network can focus on discovering these patterns within the image. 

In this notebook, we will see how CNNs perform as compared to FC nerworks.

In [1]:
from res.plot_lib import *
set_default() # setting the default plot style

In [2]:
# importing dependencies
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import dataset, DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy

In [3]:
# function to calculate the number of parameters in the model
def count_parameters(model):
    params = 0
    for p in list(model.parameters()):
        params += p.nelement()
    return params

In [4]:
# selecting the default device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [5]:
# creating the dataloaders to feed data to the models
train_loader = DataLoader(
    datasets.MNIST("E-Learning/NYU-DL/data/MNIST", train=True, download=True, 
                        transform= transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=64,
    shuffle=True,
)

validation_loader = DataLoader(
    datasets.MNIST("E-Learning/NYU-DL/data/MNIST", train=False, download=True, 
                        transform= transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=256,
    shuffle=False,
)

## Modeiling
---
### FC Model:

In [9]:
class FCModel(nn.Module):
    def __init__(self, input_size, n_hidden, output_size):
        super(FCModel, self).__init__()
        self.input_size = input_size
        self.n_hidden = n_hidden
        self.output_size = output_size 
        self.network = nn.Sequential(
            nn.Linear(self.input_size, self.n_hidden),
            nn.ReLU(),
            nn.Linear(self.n_hidden, self.n_hidden),
            nn.ReLU(),
            nn.Linear(self.n_hidden, self.output_size),
            nn.LogSoftmax(dim=1),
        )
        
    def forward(self, x):
        x = x.view(-1, self.input_size)
        return self.network(x)

### CNN Model:

In [23]:
class CNNModel(nn.Module):
    def __init__(self, n_channels, output_size):
        super(CNNModel, self).__init__()
        self.n_channels = n_channels
        self.output_size = output_size
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=n_channels//2, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=n_channels//2, out_channels=n_channels, kernel_size=5)
        self.fc1 = nn.Linear(n_channels*4*4, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        # input shape = m x 28 x 28 x 1
        x = self.conv1(x) # m x 24 x 24 x n_channels//2
        x = F.relu(x)
        x = nn.MaxPool2d(kernel_size=2)(x) # m x 12 x 12 x n_channels//2
        x = self.conv2(x) # m x 8 x 8 x n_channels
        x = F.relu(x)
        x = nn.MaxPool2d(kernel_size=2)(x) # 4 x 4 x n_channels
        x = nn.Flatten()(x) # m x 4 * 4 * n_channels
        x = self.fc1(x) # m x 50
        x = F.relu(x)
        x = self.fc2(x) # m x 10
        x = F.log_softmax(x, dim=1)
        return x

Now that we have defined the models, let us define the training and validation functions.

In [24]:
model = CNNModel(n_channels=32, output_size=10)
for batch in train_loader:
    print(model(batch[0]))
    break

tensor([[-2.1084, -2.1482, -2.2663, -2.4473, -2.3575, -2.4634, -2.3264, -2.3007,
         -2.3512, -2.3151],
        [-2.0973, -2.0880, -2.2300, -2.4809, -2.3681, -2.4867, -2.3412, -2.3418,
         -2.3043, -2.3747],
        [-2.1183, -2.1165, -2.2406, -2.4180, -2.4426, -2.4634, -2.3429, -2.2858,
         -2.3568, -2.3098],
        [-2.1259, -2.1350, -2.2064, -2.4252, -2.3436, -2.4368, -2.3648, -2.3365,
         -2.3610, -2.3487],
        [-2.1035, -2.1315, -2.2470, -2.4165, -2.3327, -2.4677, -2.3589, -2.3151,
         -2.3335, -2.3826],
        [-2.1600, -2.1161, -2.2267, -2.4199, -2.3743, -2.4481, -2.3219, -2.3144,
         -2.3649, -2.3335],
        [-2.1113, -2.0821, -2.2036, -2.4475, -2.4104, -2.4816, -2.3302, -2.3681,
         -2.3516, -2.3248],
        [-2.1338, -2.1431, -2.2423, -2.4003, -2.3589, -2.4117, -2.3324, -2.2884,
         -2.4191, -2.3460],
        [-2.1546, -2.1145, -2.2247, -2.4170, -2.3682, -2.4559, -2.3371, -2.2945,
         -2.3671, -2.3489],
        [-2.1626, -